*Основные цели этого задания:*
*   Научиться генерировать негативы.
*   Научиться настраивать алгоритмы коллаборативной фильтрации.

Задача: Научиться рекомендовать пользователям фильмы на основе факта просмотра фильмов пользователями. 

In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds

pd.options.display.max_colwidth = 2000
pd.options.display.float_format = '{:.2f}'.format

*1. Для решения задачи будем использовать те же данные, которые были использованы в скринкастах:*
*   MovieLens — источник данных.
*   Предобработанные для обучения данные: история проставления оценок фильмам — ratings_df_sample_2.csv.

In [3]:
ratings = pd.read_csv('ratings_df_sample_2.csv')
movies = pd.read_csv('movies.csv')

OSError: [Errno 22] Invalid argument: 'D:\\Аня\\Мои документы\x01 tData\x02.7\ratings_df_sample_2.csv'

In [ ]:
ratings=(
    ratings
    .merge(movies, on='movieId')
)

In [ ]:
ratings.shape

hh   

*2. Предположим, постановка рейтинга — обязательное по итогам просмотра фильмов действие. Основываясь на этом, сгенерируйте новый целевой признак «факт просмотра фильма пользователем», который будет равен 1 для всех пар пользователь * фильм из подгруженного датасета.*

*3. А откуда взять «нолики»? В наших данных есть только пары пользователь * фильм, в которых пользователь точно смотрел фильм. Но для обучения модели нужны так называемые «негативы», то есть, пары, где пользователь фильм не смотрел. На практике приходится сталкиваться с необходимостью генерировать их вручную, давайте потренируемся это делать.*
*   Сначала найдите уникальные id всех пользователей и уникальные id всех фильмов.
*   С помощью функции random.choice (документация) сгенерируйте случайные пары пользователь * фильм
*   Поскольку среди сгенерированных пар могут быть и такие, что пользователь в них уже смотрел фильм, сгенерируйте побольше пар, например, удвоенное количество строк из источника. Это может занять пару минут.
*   Среди сгенерированных пар могут быть и дубликаты, удалите их.
*   Оставьте среди сгенерированных пар только те, в которых пользователь фильм не смотрел.
*   Возможно, пар получилось больше, чем нужно, выберите из них столько, сколько у нас строк в исходных данных.
*   Добавьте очищенные сгенерированные пары к исходным данным. Значение целевого признака в них будет равно нулю. Убедитесь, что у вас не появились дубликаты в датасете.

*4. Подготовьте датасет к обучению: отделите тестовую часть от тренировочной.*

*5. Обучите dummy-model. Пусть она будет возвращать случайную вероятность принадлежности классу 1. Для этого можете использовать функцию random.random (документация). Оцените ее качество какой-то метрикой на свой вкус. Необходимо прогнозировать именно вероятность, чтобы была возможность ранжировать по ней варианты для рекомендации лучшего контента пользователю.*

*6. Реализуйте три алгоритма коллаборативной фильтрации: user-, item-based и алгоритм на основе матричной факторизации. Оцените их качество и адекватность. Если качество недостаточно хорошее, попробуйте варьировать параметры: количество похожих пользователей/фильмов, количество элементов в матрицах при матричном разложении.*

*7. Опишите вывод, содержащий информацию о том, какой алгоритм проявил себя лучше всего.*